In [2]:
f = open("grada2_input.txt","w")
for i in range(600):
    f.write("grd_2/%s.jpg\n"%str(i+1).zfill(5))

f.close()

In [23]:
def boxes_match(box1,box2,iou_threshold=0.4):
    
    if(box1[1]!=box2[1] or box1[7]==1 or box2[7]==1):
        return False
    
    return iou(box1[3:7],box2[3:7])>0.4

def boxes_equal(box1,box2,iou_threshold=0.6):
   
    if(box1[1]!=box2[1] or box1[2]!=box2[2]):
        return False
    
    return iou(box1[3:7],box2[3:7])==1
def box_matches_list(box1,list_boxes):
    index =-1
    if(len(list_boxes)==0):
        return index
    match_list = []
    box_index =0
    
    for box in list_boxes:
       
        if(box.shape[0]>0):
            
            if(box.shape[0]==1):
                
                if(boxes_match(box[0],box1)):
                    match_list.append(box_index)
                    
            elif(box.shape[0]==8):
                
                if(boxes_match(box,box1)):
                    
                    match_list.append(box_index)
        box_index+=1
    
    if(len(match_list)== 0):
        return index
    max_match_index = 0
    max_match_iou = 0
    for i in match_list:
        cur_iou = iou(box1[3:7],list_boxes[i][3:7])
        
        if(cur_iou>max_match_iou):
            max_match_iou=cur_iou
            max_match_index =i
    list_boxes[max_match_index][7] = 1
    box1[7] = 1
    
    return max_match_index

def box_in_list(box1,list_boxes):
    if(len(list_boxes)==0):
        return -1
    index=0
    for box in list_boxes:
        if(box.shape[0]>0):
            if(box.shape[0]==1):
                if(boxes_match(box[0],box1)):
                    return index
            elif(box.shape[0]==7):
                if(boxes_match(box,box1)):
                    return index
        index+=1
    return -1
def center(box):
    return np.array([(box[3]+box[5])/2,(box[4]+box[6])/2],np.float32)

In [28]:
with open('yolo_modd_baseline.json', encoding='utf-8') as data_file:
    data = json.loads(data_file.read())
print(data[0])
detections={}
for item in data:
    
    file_name = "modd/%s.jpg"%str(item['image_id']).zfill(5)
    if(file_name in detections):
        detections[file_name].append(list([item['category_id'],item['score'],item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3]]))
    else:
        detections[file_name]=[]
        detections[file_name].append(list([item['category_id'],item['score'],item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3]]))
print(detections)

{'image_id': 56, 'category_id': 1, 'bbox': [1.0, 184.0, 44.0, 116.0], 'score': 0.589566}
{'modd/00056.jpg': [[1, 0.589566, 1.0, 184.0, 44.0, 116.0]], 'modd/00057.jpg': [[1, 0.79956, 1.0, 180.0, 55.0, 118.0]], 'modd/00058.jpg': [[1, 0.666275, 0.0, 178.0, 62.0, 125.0]], 'modd/00059.jpg': [[1, 0.655662, 1.0, 181.0, 69.0, 125.0]], 'modd/00060.jpg': [[1, 0.670983, 1.0, 191.0, 72.0, 128.0]], 'modd/00061.jpg': [[1, 0.631756, 0.0, 195.0, 77.0, 128.0]], 'modd/00062.jpg': [[1, 0.811305, 0.0, 224.0, 92.0, 126.0]], 'modd/00063.jpg': [[1, 0.969736, 2.0, 205.0, 105.0, 134.0]], 'modd/00064.jpg': [[1, 0.974496, 2.0, 205.0, 109.0, 134.0]], 'modd/00065.jpg': [[1, 0.977615, 3.0, 205.0, 110.0, 134.0]], 'modd/00066.jpg': [[1, 0.967917, 3.0, 206.0, 112.0, 134.0]], 'modd/00067.jpg': [[1, 0.984028, 2.0, 200.0, 116.0, 143.0]], 'modd/00068.jpg': [[1, 0.984994, 0.0, 201.0, 122.0, 141.0]], 'modd/00069.jpg': [[1, 0.980564, 0.0, 202.0, 128.0, 139.0]], 'modd/00070.jpg': [[1, 0.9549, 0.0, 207.0, 131.0, 131.0]], 'modd

In [21]:
f = open("yolo_output.txt","r")
line = f.readline()
detections={}
comps = []
while(line):
    
    line = line.replace("\n", "")
    comps = line.split(",")
    print(comps)
    if(comps[0] in detections):
        detections[comps[0]].append(list(map(float, comps[2:])))
    else:
        detections[comps[0]]=[]
        detections[comps[0]].append(list(map(float, comps[2:])))
    line=f.readline()
f.close()
print(detections)

['modd/00056.jpg', 'boat', '8', '0.589566', '1', '184', '45', '300']
['modd/00057.jpg', 'boat', '8', '0.799560', '1', '180', '56', '298']
['modd/00058.jpg', 'boat', '8', '0.666275', '0', '178', '62', '303']
['modd/00059.jpg', 'boat', '8', '0.655662', '1', '181', '70', '306']
['modd/00060.jpg', 'boat', '8', '0.670983', '1', '191', '73', '319']
['modd/00061.jpg', 'boat', '8', '0.631756', '0', '195', '77', '323']
['modd/00062.jpg', 'boat', '8', '0.811305', '0', '224', '92', '350']
['modd/00063.jpg', 'boat', '8', '0.969736', '2', '205', '107', '339']
['modd/00064.jpg', 'boat', '8', '0.974496', '2', '205', '111', '339']
['modd/00065.jpg', 'boat', '8', '0.977615', '3', '205', '113', '339']
['modd/00066.jpg', 'boat', '8', '0.967917', '3', '206', '115', '340']
['modd/00067.jpg', 'boat', '8', '0.984028', '2', '200', '118', '343']
['modd/00068.jpg', 'boat', '8', '0.984994', '0', '201', '122', '342']
['modd/00069.jpg', 'boat', '8', '0.980564', '0', '202', '128', '341']
['modd/00070.jpg', 'boat', 

In [22]:
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

In [29]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
import numpy as np
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =70
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 640
frame_height = 464
print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]
out_tracking = cv.VideoWriter('yolo_keypointflow_modd.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code
print("prev_count   current_count   entered   exited")
multiplier=0
cc=0
prev_frame=None

while frameCount<641:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
 
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    if(frameCount<0):
        continue
    elif started==False:
        start_time = time.time()
        started=True
    # resize the frame, convert it to grayscale, and blur it
    
    
    
        
    # loop over the contours
    preds = []
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=1):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)
   
    for track in tracks:
        track[7]=0
    #print(len(prev_pred),len(temp),len(entered_pred),len(exited_pred))
    
    for pred in preds:
        
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                
                multiplier +=1
                
             
            del tracks[index]
            
    
            pred[2] += multiplier*step
            
            pred[0] = multiplier
            #print(pred)
            
            tracks.append(pred)
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
            
          
            temp_pred[0] = multiplier
            temp_pred[2] = 1
            
            tracks.append(temp_pred)
            
            cc = frameCount
    
    for track in tracks:
        if(track[7]!=1):
            if(box_matches_list(track,preds)==-1):
                
                #print('not found in current frame')
                track[0]-=1
                #print('multiplier decreased to ',track[0])
                track[2]+=(step*track[0])
                frame_grey = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
                prev_frame_grey = cv.cvtColor(prev_frame,cv.COLOR_BGR2GRAY)
                mask = np.zeros(frame_grey.shape, dtype = "uint8")


                cv.rectangle(mask, (int(track[3]), int(track[4])), (int(track[5]), int(track[6])), (255, 255, 255), -1)
                p0 = cv.goodFeaturesToTrack(prev_frame_grey, mask = mask, **feature_params)
                if(not p0 is None ):
                    p1, st, err = cv.calcOpticalFlowPyrLK(prev_frame_grey, frame_grey, p0, None, **lk_params)

                    average_flow = np.average(p1-p0,0)[0]
                else:
                    average_flow=[0,0]
                xmin = int(track[3])
                ymin = int(track[4])
                xmax =int(track[5] )
                ymax =int(track[6])
                flow_x_box_coords = int(average_flow[0])
                flow_y_box_coords = int(average_flow[1])
                track[3] += flow_x_box_coords
                track[5]+=flow_x_box_coords
                track[4]+=flow_y_box_coords
                track[6]+=flow_y_box_coords
            
    
    to_display = [track for track in tracks if track[2]>0.4]
    
    for box in to_display:
        
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
        
        xmin = int(box[3] )
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmin+xmax),int(ymin+ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
    #print('preds%d'%len(preds))
    #print('tracks%d'%len(tracks))
    #print('displayed%d'%len(to_display))
    # if the `q` key is pressed, break from the lop
    #plt.figure()
    #plt.imshow(frame)
    cv.imwrite("one.jpg",frame)
    out_tracking.write(frame)
    prev_frame=frame
    frameCount+=1
# cleanup the camera and close any open windows
elapsed_time = time.time() - start_time
print(elapsed_time)

cv.destroyAllWindows()

out_tracking.release()

print(total_objects_tracking)
with open('yolo_keypoint_modd.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

640
464
prev_count   current_count   entered   exited
21.392503023147583
93


In [ ]:
cv.rectangle??

In [19]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
import numpy as np
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =70
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 640
frame_height = 464
print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]
out_tracking = cv.VideoWriter('ssd_300_kalman_tracking.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code
print("prev_count   current_count   entered   exited")
multiplier=0

cc=0
prev_frame=None
kalman = cv.KalmanFilter(4,2)
kalman.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

kalman.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

kalman.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.01
measurement = np.array((2,1), np.float32)
prediction = np.zeros((2,1), np.float32)
while frameCount<641:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
 
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    if(frameCount<0):
        continue
    elif started==False:
        start_time = time.time()
        started=True
    # resize the frame, convert it to grayscale, and blur it
    
    
    
        
    # loop over the contours
    preds = []
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)
   
    for track in tracks:
        track[7]=0
    #print(len(prev_pred),len(temp),len(entered_pred),len(exited_pred))
    prediction = kalman.predict();
    
    for pred in preds:
        
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            kalman.correct(center(pred));

          
            prediction = kalman.predict();
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                
                multiplier +=1
                
             
            del tracks[index]
            
    
            pred[2] += multiplier*step
            
            pred[0] = multiplier
            #print(pred)
            
            tracks.append(pred)
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
            
          
            temp_pred[0] = multiplier
            temp_pred[2] = 1
            
            tracks.append(temp_pred)
            
            cc = frameCount
    
    for track in tracks:
        if(track[7]!=1):
            if(box_matches_list(track,preds)==-1):
                
                predicted_center= np.array([prediction[0][0],prediction[1][0]],np.float32)
            
                
                offset = predicted_center- center(track)
                
                track[3] += offset[0]
                track[5]+=offset[0]
                track[4]+=offset[1]
                track[6]+=offset[1]
            
    
    to_display = [track for track in tracks if track[2]>0.4]
    
    for box in to_display:
        
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
        
        xmin = int(box[3] )
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv.circle(frame,(prediction[0][0],prediction[1][0]),5,(255,0,0),2)
        
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
    #print('preds%d'%len(preds))
    #print('tracks%d'%len(tracks))
    #print('displayed%d'%len(to_display))
    # if the `q` key is pressed, break from the lop
    #plt.figure()
    #plt.imshow(frame)
    cv.imwrite("one.jpg",frame)
    out_tracking.write(frame)
    prev_frame=frame
    frameCount+=1
# cleanup the camera and close any open windows
elapsed_time = time.time() - start_time
print(elapsed_time)

cv.destroyAllWindows()

out_tracking.release()

print(total_objects_tracking)
with open('ssd300_kalman_tracking.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

640
464
prev_count   current_count   entered   exited
12.180943965911865
571


In [125]:
print(center(np.array([1,1,1,20,50,80,150],np.float32)))

[ 50. 100.]


In [2]:
import cv2 as cv
cv.KalmanFilter??

In [18]:
# import the necessary packages
from imutils.video import VideoStream
import argparse
import datetime
import imutils
import time
import cv2 as cv
import time
import json
import numpy as np
from bounding_box_utils.bounding_box_utils import iou
feature_params = dict( maxCorners = 25,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
no_tracking_res = [] 
tracking_res = []

# initialize the first frame in the video stream
firstFrame = None
frameCount =70
step = 0.05
total_objects_no_tracking=0
total_objects_tracking=0
frame_width = 640
frame_height = 464
print(frame_width)
print(frame_height)
preds = []
pred = None
tracks=[]
out_tracking = cv.VideoWriter('ssd_300_kalman_tracking.avi',cv.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))
started = False
# your code
print("prev_count   current_count   entered   exited")
multiplier=0

cc=0
prev_frame=None
kalman_min = cv.KalmanFilter(4,2)
kalman_min.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

kalman_min.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

kalman_min.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.01
kalman_max = cv.KalmanFilter(4,2)
kalman_max.measurementMatrix = np.array([[1,0,0,0],
                                     [0,1,0,0]],np.float32)

kalman_max.transitionMatrix = np.array([[1,0,1,0],
                                    [0,1,0,1],
                                    [0,0,1,0],
                                    [0,0,0,1]],np.float32)

kalman_max.processNoiseCov = np.array([[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,0],
                                   [0,0,0,1]],np.float32) * 0.01
measurement = np.array((2,1), np.float32)
prediction = np.zeros((2,1), np.float32)
while frameCount<641:
    # grab the current frame and initialize the occupied/unoccupied
    # text
    frame = cv.imread('../modd/%s.jpg'%str(frameCount+1).zfill(5))
    
 
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        break
    
    if(frameCount<0):
        continue
    elif started==False:
        start_time = time.time()
        started=True
    # resize the frame, convert it to grayscale, and blur it
    
    
    
        
    # loop over the contours
    preds = []
    if 'modd/%s.jpg'%str(frameCount+1).zfill(5) in detections:
        
        for box in detections['modd/%s.jpg'%str(frameCount+1).zfill(5)]:
           
            if(box[0]!=4):

                continue
            temp_pred = np.insert(box,0,0)
            temp_pred=np.insert(temp_pred,7,0)
            preds.append(temp_pred)
   
    for track in tracks:
        track[7]=0
    #print(len(prev_pred),len(temp),len(entered_pred),len(exited_pred))
    prediction_min = kalman_min.predict();
    prediction_max = kalman_max.predict();
    for pred in preds:
        
        index = box_matches_list(pred,tracks)
        if(index>-1):
            
            multiplier = tracks[index][0]
            
        
            kalman_min.correct(np.array([box[2],box[3]],np.float32));
            kalman_max.correct(np.array([box[4],box[5]],np.float32));
          
            prediction = kalman.predict();
            org_conf =tracks[index][2]
            if(multiplier<0):
                multiplier=1
            else:
                
                multiplier +=1
                
             
            del tracks[index]
            
    
            pred[2] += multiplier*step
            
            pred[0] = multiplier
            #print(pred)
            
            tracks.append(pred)
        else:
            
            temp_pred = np.copy(pred)
           
            multiplier = 1
            
          
            temp_pred[0] = multiplier
            temp_pred[2] = 1
            
            tracks.append(temp_pred)
            
            cc = frameCount
    
    for track in tracks:
        if(track[7]!=1):
            if(box_matches_list(track,preds)==-1):
                
                
                track[3] = prediction_min[0][0]
                track[5]=prediction_min[1][0]
                track[4]=prediction_max[0][0]
                track[6]=prediction_max[1][0]
            
    
    to_display = [track for track in tracks if track[2]>0.4]
    
    for box in to_display:
        
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
        
        xmin = int(box[3] )
        ymin = int(box[4])
        xmax =int(box[5])
        ymax =int(box[6])
        cv.rectangle(frame, (int(xmin), int(ymin)), (int(xmax),int(ymax)), (0, 255, 0), 2)
        cv.putText(frame,'{}: {:.2f}'.format(classes[int(box[1])], box[2]), (xmin, ymin),cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        print('detection:')
        print(xmin,ymin,xmax,ymax)
        print('kalman:')
        
        
        cv.circle(frame,(prediction_min[0][0],prediction_min[1][0]),5,(255,0,0),2)
        cv.circle(frame,(prediction_max[0][0],prediction_max[1][0]),5,(255,0,0),2)
        total_objects_tracking+=1
        tracking_res.append({"image_id" : frameCount, "category_id" : 1, "bbox" : [float(xmin),float(ymin),float(xmax-xmin),float(ymax-ymin)], "score" : int(box[1])})
    #print('preds%d'%len(preds))
    #print('tracks%d'%len(tracks))
    #print('displayed%d'%len(to_display))
    # if the `q` key is pressed, break from the lop
    #plt.figure()
    #plt.imshow(frame)
    cv.imwrite("one.jpg",frame)
    out_tracking.write(frame)
    prev_frame=frame
    frameCount+=1
# cleanup the camera and close any open windows
elapsed_time = time.time() - start_time
print(elapsed_time)

cv.destroyAllWindows()

out_tracking.release()

print(total_objects_tracking)
with open('ssd300_kalman_tracking.json', 'w') as outfile:  
    json.dump(tracking_res, outfile)

640
464
prev_count   current_count   entered   exited
detection:
-17 172 156 340
kalman:
detection:
-24 160 162 340
kalman:
detection:
-12 130 157 335
kalman:
detection:
-12 126 161 329
kalman:
detection:
-10 126 163 328
kalman:
detection:
-8 133 165 325
kalman:
detection:
-13 128 169 329
kalman:
detection:
-12 148 169 330
kalman:
detection:
-15 146 172 341
kalman:
detection:
-13 194 179 368
kalman:
detection:
-9 204 181 372
kalman:
detection:
-8 205 186 365
kalman:
detection:
-10 208 187 366
kalman:
detection:
-13 196 195 352
kalman:
detection:
-13 171 199 342
kalman:
detection:
-13 151 204 334
kalman:
detection:
-10 139 208 333
kalman:
detection:
-7 149 212 333
kalman:
detection:
-6 135 220 334
kalman:
detection:
-7 149 230 342
kalman:
detection:
-11 154 236 335
kalman:
detection:
-11 158 238 335
kalman:
detection:
-7 169 234 351
kalman:
detection:
-7 174 230 356
kalman:
detection:
-12 193 226 353
kalman:
detection:
-12 179 232 323
kalman:
detection:
-16 167 234 314
kalman:
detection

detection:
165 304 166 265
kalman:
detection:
133 150 320 257
kalman:
detection:
153 310 161 263
kalman:
detection:
146 139 325 245
kalman:
detection:
149 318 152 256
kalman:
detection:
152 143 327 248
kalman:
detection:
150 323 148 252
kalman:
detection:
145 141 336 249
kalman:
detection:
147 331 143 249
kalman:
detection:
147 145 335 255
kalman:
detection:
146 336 142 250
kalman:
detection:
148 143 317 246
kalman:
detection:
145 331 141 247
kalman:
detection:
145 331 141 247
kalman:
detection:
144 334 139 246
kalman:
detection:
144 334 139 246
kalman:
detection:
147 139 313 246
kalman:
detection:
144 336 138 245
kalman:
detection:
144 336 138 245
kalman:
detection:
151 144 310 252
kalman:
detection:
147 320 140 248
kalman:
detection:
147 320 140 248
kalman:
detection:
147 320 140 248
kalman:
detection:
147 320 139 248
kalman:
detection:
147 320 139 248
kalman:
detection:
147 320 139 248
kalman:
detection:
150 155 294 265
kalman:
detection:
147 319 138 247
kalman:
detection:
147 319 1

detection:
175 316 157 252
kalman:
detection:
175 316 157 252
kalman:
detection:
175 316 157 252
kalman:
detection:
175 316 157 252
kalman:
detection:
175 316 157 252
kalman:
detection:
175 316 157 252
kalman:
detection:
175 316 157 252
kalman:
detection:
178 149 308 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 313 152 250
kalman:
detection:
177 154 312 245
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
178 314 152 247
kalman:
detection:
179 315 151 245
kalman:
detection:
179 315 151 245
kalman:
detection:
179 315 151 245
kalman:
detection:
179 315 151 245
kalman:
detection:
179 315 1

detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
203 330 156 257
kalman:
detection:
206 141 344 246
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
204 330 155 257
kalman:
detection:
205 330 155 258
kalman:
detection:
205 330 155 258
kalman:
detection:
205 330 155 258
kalman:
detection:
205 330 155 258
kalman:
detection:
205 330 155 258
kalman:
detection:
205 330 155 258
kalman:
detection:
205 330 1

detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
346 313 172 223
kalman:
detection:
225 168 348 251
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 168 249
kalman:
detection:
232 347 1

...

detection:
515 556 207 255
kalman:
detection:
515 556 207 255
kalman:
detection:
515 556 207 255
kalman:
detection:
515 556 207 255
kalman:
detection:
419 205 594 275
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 205 262
kalman:
detection:
479 570 2

detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 219 286
kalman:
detection:
448 600 2

detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 207 278
kalman:
detection:
477 637 2

578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
578 595 198 256
kalman:
detection:
492 223 664 307
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalman:
detection:
588 602 201 261
kalm

detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 222 309
kalman:
detection:
524 665 2